In [7]:
# measurement

import serial
import time

ser = serial.Serial('COM8', 9600)
time.sleep(2)

measurements = [] # 측정한 전체 데이터
current_data = [] # 측정할 때 사용할 딕셔너리
measurement_started = False
saving = False
try:
    while True:
        if ser.inWaiting() > 0:

            line = ser.readline().decode('utf-8').rstrip() # 데이터 읽기

            if "Arduino" in line:
                print(line)
            
            if "Stop" in line:
                print(line)
                ser.close()
                break
            
            if "Start measurement" in line:
                print(line)
                measurement_started = True
                current_measurement = {"time": time.time(), "pin1": [], "pin2": []}  # 새로운 측정을 위해 초기화
            elif "End measurement" in line:
                print(line)
                print(f"measurement for {time.time() - current_measurement['time']}")
                measurement_started = False
                current_measurement["time"] = time.time() - current_measurement["time"]  # 측정 시간 계산
                measurements.append(current_measurement)  # 현재 측정 데이터를 전체 리스트에 추가
                print(measurements)
            elif measurement_started:
                if line.startswith("pin1: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin1"].append(value)
                elif line.startswith("pin2: "):
                    value = int(line.split(": ")[1])
                    current_measurement["pin2"].append(value)
            
except KeyboardInterrupt:
    ser.close()
    print("Serial port closed.")
    print("Collected data:", measurements)
            

--------Arduino ON--------
Start measurement
End measurement. (1.51sec)
measurement for 1.8306736946105957
[{'time': 1.8306736946105957, 'pin1': [6, 12, 7, 5, 6, 19, 16, 8, 7, 15, 10], 'pin2': [16, 13, 8, 9, 10, 33, 32, 15, 12, 8, 6]}]
Start measurement
End measurement. (1.51sec)
measurement for 1.8307108879089355
[{'time': 1.8306736946105957, 'pin1': [6, 12, 7, 5, 6, 19, 16, 8, 7, 15, 10], 'pin2': [16, 13, 8, 9, 10, 33, 32, 15, 12, 8, 6]}, {'time': 1.8307108879089355, 'pin1': [8, 7, 9, 13, 7, 9, 12, 14, 10, 10, 6, 11], 'pin2': [16, 15, 13, 10, 10, 28, 15, 6, 13, 7, 12, 8, 5]}]
Start measurement
End measurement. (1.31sec)
measurement for 1.6282081604003906
[{'time': 1.8306736946105957, 'pin1': [6, 12, 7, 5, 6, 19, 16, 8, 7, 15, 10], 'pin2': [16, 13, 8, 9, 10, 33, 32, 15, 12, 8, 6]}, {'time': 1.8307108879089355, 'pin1': [8, 7, 9, 13, 7, 9, 12, 14, 10, 10, 6, 11], 'pin2': [16, 15, 13, 10, 10, 28, 15, 6, 13, 7, 12, 8, 5]}, {'time': 1.6282081604003906, 'pin1': [8, 13, 6, 5, 8, 17, 9, 11, 5

In [54]:
# 예외 떴을 때 사용 (시리얼 통신 끊기)
ser.close()

In [13]:
# initializing CustomDataset
import torch
from torch.utils.data import Dataset, ConcatDataset, DataLoader
from custom_dataset import CustomDataset
file_name = 'bilabial' 

In [8]:
# save Dataset (기존 데이터셋이 있으면 ConcatDataset 인스턴스로 변경 후 데이터 추가 후 저장, 없으면 새 데이터셋(CustomDataset)을 생성)
new_dataSet = CustomDataset(measurements)

try:
    existing_dataSet = torch.load(file_name + '.pth')
    if isinstance(new_dataSet, CustomDataset):
        combined_dataSet = ConcatDataset([existing_dataSet, new_dataSet])
        print('기존에 데이터셋이 있으므로 기존 데이터셋에 데이터를 추가합니다.')
        torch.save(combined_dataSet, file_name + '.pth')
    else:
        combined_dataSet = new_dataSet
        print('기존에 데이터셋은 존재하나 CustomDataset이 아니므로 스킵합니다.')
except FileNotFoundError:
    combined_dataSet = new_dataSet
    print('기존에 데이터셋이 존재하지 않으므로 새로운 데이터셋을 추가합니다.')
    torch.save(combined_dataSet, file_name + '.pth')

기존에 데이터셋이 있으므로 기존 데이터셋에 데이터를 추가합니다.


In [14]:
# ConcatDataset -> CustomDataset으로 변환
from torch.utils.data.dataset import ConcatDataset
loaded_dataset = torch.load(file_name + '.pth')
print(loaded_dataset)
if isinstance(loaded_dataset, CustomDataset):
    print('현재 파일은 CustomDataset의 인스턴스입니다.')
else:
    print('현재 파일은 CustomDataset의 인스턴스가 아닙니다. CustomDataset으로 변환합니다.')
    # ConcatDataset에 포함된 모든 데이터셋의 요소를 모두 모아 하나의 리스트로 만듬
    all_data = []
    for data_tuple in loaded_dataset:
        data_point = {'time': data_tuple[0], 'pin1': data_tuple[1], 'pin2': data_tuple[2]}
        all_data.append(data_point)
    
    # 새로운 CustomDataset 객체 생성 및 저장
    custom_dataset = CustomDataset(all_data)
    torch.save(custom_dataset, file_name + '.pth')

현재 파일은 CustomDataset의 인스턴스입니다.


In [10]:
# load Dataset
from torch.nn.utils.rnn import pad_sequence
from padding import PaddingCollate
loaded_dataset = torch.load(file_name + '.pth')

# DataLoader에 로드된 데이터셋 사용
dataloader = DataLoader(loaded_dataset, batch_size=100, shuffle=True, collate_fn=PaddingCollate())
for time, pin1, pin2 in dataloader:
    print(time.size())
    print("Time:", time)
    print("Pin1:", pin1.int())
    print("Pin2:", pin2.int())


torch.Size([50])
Time: tensor([1.5256, 1.6272, 1.6266, 2.1346, 1.5256, 1.6271, 1.6272, 1.7286, 1.6272,
        1.5251, 1.5256, 1.6258, 1.5271, 1.6270, 1.5257, 1.6271, 1.6272, 1.5259,
        1.5256, 1.7282, 1.7282, 1.5255, 1.6261, 1.5258, 1.6282, 1.5262, 1.6266,
        1.8307, 1.5258, 1.5254, 1.5260, 1.6267, 2.0321, 1.8309, 1.6271, 1.5248,
        1.5258, 1.6269, 1.6277, 1.7280, 1.6267, 2.0340, 1.8307, 1.6274, 1.7286,
        1.8304, 1.8311, 1.5260, 1.6270, 1.5255])
Pin1: tensor([[13,  9, 14, 20, 19, 24,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9, 14, 39,  7,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 8,  9, 19, 81, 32,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 6,  7,  6,  6, 22,  7, 38, 20, 15, 11, 10,  7, 15, 14, 15],
        [11, 21, 10, 12,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 9,  5,  5,  9, 17, 14,  7, 12, 24,  0,  0,  0,  0,  0,  0],
        [ 5, 25, 39, 62,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [19,  9, 11, 18, 15, 21,